In [36]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from ipywidgets import interact

In [37]:
# Define global variables for max and min values
MIN_LIFETIME = 1
MAX_LIFETIME = 20
MIN_INF_FREQ = 1
MAX_INF_FREQ = 3650
MIN_CORE_FREQ = 1000
MAX_CORE_FREQ = 100000


In [38]:
# -------------------------
# 1) Define Internal Data
# -------------------------
system_specs = {
    "Serv": {"area": 2.93, "power": 17.7451},
    "Qerv": {"area": 3.68, "power": 21.0650},
    "Herv": {"area": 4.50, "power": 24.9934},
}

# In number of cycles
execution_time = {
    "Food Spoilage Detection":  {"Serv": 6171.3, "Qerv": 1963.5, "Herv": 1256.9},
    "Cardiotocography": {"Serv": 415615.7, "Qerv": 131434.4, "Herv": 83432.6},
    "Water Quality Detection": {"Serv": 4376.9, "Qerv": 1401.05, "Herv": 900.65},
    "Air Pollution Monitoring": {"Serv": 1460624.9, "Qerv": 461267.6, "Herv": 292341.2},
    "Odor Detection": {"Serv": 5359.7, "Qerv": 1708.65, "Herv": 1094.65},
    "Smart Irrigation": {"Serv": 1130791.65, "Qerv": 357050.15, "Herv": 226125.75},
}

rf_sram = 128 # Implementing register file in SRAM for space-saving


# Define SRAM and LPROM constants (in bits)
sram_power = 0.0018
sram_area = 0.00176
lprom_power = 0 # negligible
lprom_area = 0.000359
sram_overhead_power = 0.0002 # per bit
sram_overhead_area = 0.0003 # per bit

# In bytes
sram = {
    "Food Spoilage Detection": 49,
    "Cardiotocography": 590,
    # "Arrhythmia Detection": 4170,
    "Water Quality Detection": 13,
    # "Smart HVAC Monitoring": 63,
    # "Gesture Recognition": 40033,
    "Air Pollution Monitoring": 591,
    "Odor Detection": 21,
    "Smart Irrigation": 77,
    # "Animal Tracking": 39193 # This one could be larger
}

# In bytes
lprom = {
    "Food Spoilage Detection": 1676,
    "Cardiotocography": 4256,
    # "Arrhythmia Detection": 3970,
    "Water Quality Detection": 368,
    # "Smart HVAC Monitoring": 102432,
    # "Gesture Recognition": 200508,
    "Air Pollution Monitoring": 3373,
    "Odor Detection": 1188,
    "Smart Irrigation": 1348,
    # "Animal Tracking": 3448
}

# Define carbon intensity values for each power source
# gCO2 eq / mWs
carbon_intensity_values = {
    "Coal": "0.000000228",
    "Natural Gas": "0.000000136",
    "Nuclear": "0.00000000333",
    "Wind (Onshore)": "0.00000000306",
    "Solar PV (Utility)": "0.0000000133",
    "Global Avg (2018)": "0.000000132",
    "US Avg (2020)": "0.000000108",
    "Custom": ""
}

embodied_values = {
    "Food Spoilage Detection": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0.87, "Carbon Footprint Max (gCO2e)": 1.17},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0.93, "Carbon Footprint Max (gCO2e)": 1.25},
        "Herv": {"Carbon Footprint Min (gCO2e)": 1.0, "Carbon Footprint Max (gCO2e)": 1.34},
    },
    "Cardiotocography": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 2.2, "Carbon Footprint Max (gCO2e)": 2.96},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 2.26, "Carbon Footprint Max (gCO2e)": 3.04},
        "Herv": {"Carbon Footprint Min (gCO2e)": 2.32, "Carbon Footprint Max (gCO2e)": 3.13},
    },
    "Arrhythmia Detection": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 6.93, "Carbon Footprint Max (gCO2e)": 9.34},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 6.99, "Carbon Footprint Max (gCO2e)": 9.42},
        "Herv": {"Carbon Footprint Min (gCO2e)": 7.06, "Carbon Footprint Max (gCO2e)": 9.51},
    },
    "Water Quality Detection": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0.51, "Carbon Footprint Max (gCO2e)": 0.69},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0.58, "Carbon Footprint Max (gCO2e)": 0.77},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0.64, "Carbon Footprint Max (gCO2e)": 0.86},
    },
    "Smart HVAC Monitoring": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 24.44, "Carbon Footprint Max (gCO2e)": 32.92},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 24.5, "Carbon Footprint Max (gCO2e)": 33.0},
        "Herv": {"Carbon Footprint Min (gCO2e)": 24.57, "Carbon Footprint Max (gCO2e)": 33.09},
    },
    "Gesture Recognition": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 100.99, "Carbon Footprint Max (gCO2e)": 136.02},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 101.05, "Carbon Footprint Max (gCO2e)": 136.1},
        "Herv": {"Carbon Footprint Min (gCO2e)": 101.12, "Carbon Footprint Max (gCO2e)": 136.19},
    },
    "Air Pollution Monitoring": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 1.99, "Carbon Footprint Max (gCO2e)": 2.68},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 2.05, "Carbon Footprint Max (gCO2e)": 2.76},
        "Herv": {"Carbon Footprint Min (gCO2e)": 2.12, "Carbon Footprint Max (gCO2e)": 2.85},
    },
    "Odor Detection": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0.72, "Carbon Footprint Max (gCO2e)": 0.96},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0.78, "Carbon Footprint Max (gCO2e)": 1.05},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0.84, "Carbon Footprint Max (gCO2e)": 1.14},
    },
    "Smart Irrigation": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0.83, "Carbon Footprint Max (gCO2e)": 1.12},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0.89, "Carbon Footprint Max (gCO2e)": 1.2},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0.96, "Carbon Footprint Max (gCO2e)": 1.29},
    },
    "Animal Tracking": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 53.8, "Carbon Footprint Max (gCO2e)": 72.45},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 53.86, "Carbon Footprint Max (gCO2e)": 72.53},
        "Herv": {"Carbon Footprint Min (gCO2e)": 53.92, "Carbon Footprint Max (gCO2e)": 72.62},
    },
}
    

In [39]:
def compute_total_carbon(
    system: str,
    workload: str,
    lifetime_yrs: float,
    inf_freq: float, # per year
    carbon_intensity: float,
    
    core_freq: float  # New input for core frequency
):
    """
    Compute the total carbon footprint of a given system with the user-specified parameters.
    Returns (embodied_carbon, operational_carbon, total_carbon).
    """
    
    sram_bytes = sram[workload] + rf_sram
    lprom_bytes = lprom[workload]

    # 1) Calculate total area
    core_area = system_specs[system]["area"]
    device_area = core_area + (sram_area + sram_overhead_area) * sram_bytes * 8 + lprom_area * lprom_bytes * 8
    # 2) Embodied carbon (simplified)

    embodied_carbon = embodied_values[workload][system]["Carbon Footprint Max (gCO2e)"]
    # 3) Operational carbon
    #    - power_of_system + power_of_sram + power_of_lprom
    #    - execution_time * inferences
    #    - carbon intensity factor
    system_power = system_specs[system]["power"]
    total_power = system_power + (sram_power + sram_overhead_power) * sram_bytes * 8 + lprom_power * lprom_bytes * 8
    
    # Convert lifetime in years to total inferences:
    # For example: inferences_per_day * 365 * lifetime_years
    total_inferences = inf_freq * lifetime_yrs    # Execution time of each inference
    time_per_inference = execution_time[workload][system] / core_freq  # Adjusted with core frequency
    total_energy = total_power * time_per_inference * total_inferences  # (Power * Time)
    operational_carbon = total_energy * carbon_intensity
    
    total_carbon = embodied_carbon + operational_carbon

    return embodied_carbon, operational_carbon, total_carbon

def plot_carbon_vs_lifetime(workload_choice, 
                            lifetime_yrs, 
                            inf_freq, 
                            carbon_intensity,
                            core_freq):  # New input for core frequency
    """
    1) Plot total carbon vs. device lifetime for each system.
    2) Plot a 2D region map of best system choice for a grid of 
       (lifetime, inference frequency).
    """
    
    # Convert text inputs to float
    carbon_intensity = float(carbon_intensity)
    core_freq = float(core_freq)  # Convert core frequency to float

    system_colors = {'Serv': 'red', 'Qerv': 'blue', 'Herv': 'green'}  # consistent colors for each system

    # -----------------------------------------
    # Part 1: Plot total carbon vs. device lifetime
    # -----------------------------------------
    # Define the range of lifetimes
    lifetimes = np.linspace(MIN_LIFETIME, MAX_LIFETIME, 50)

    # Plot total carbon vs. device lifetime
    plt.figure(figsize=(6, 4))
    
    for system in system_specs:
        total_carbons = []
        for lifetime in lifetimes:
            _, _, total = compute_total_carbon(system, workload_choice, lifetime, inf_freq, carbon_intensity, core_freq)
            total_carbons.append(total)
        plt.plot(lifetimes, total_carbons, label=system, color=system_colors[system])

    # Add a vertical dashed line at the x value of lifetime_yrs
    plt.axvline(x=lifetime_yrs, color='gray', linestyle ='--', linewidth=1)

    plt.title(f"Total Carbon vs. Lifetime\nWorkload: {workload_choice}")
    plt.xlabel("Lifetime (years)")
    plt.ylabel("Total Carbon Footprint (gCO₂-eq)")
    plt.legend()
    plt.show()

    # -----------------------------------------
    # Part 1.1: Plot total carbon vs. inference frequency
    # -----------------------------------------
    
    inf_frequencies = np.logspace(np.log10(MIN_INF_FREQ), np.log10(MAX_INF_FREQ), 50)  # Log scale from 1 inference/day to 3650 inferences/day
    plt.figure(figsize=(6, 4))
    
    for system in system_specs:
        total_carbons = []
        valid_frequencies = []
        for freq in inf_frequencies:
            if execution_time[workload_choice][system] / core_freq <= 365 * 3600 / freq:  # Adjusted with core frequency
                _, _, total = compute_total_carbon(system, workload_choice, lifetime_yrs,
                                                   freq, carbon_intensity,
                                                   core_freq)  # Pass core frequency
                total_carbons.append(total)
                valid_frequencies.append(freq)
            else:
                break
        plt.plot(valid_frequencies, total_carbons, label=system, color=system_colors[system])

    # Add a vertical dashed line at the x value of inf_freq
    plt.axvline(x=inf_freq, color='gray', linestyle ='--', linewidth=1)

    plt.xscale('log')  # Set x axis to log scale
    plt.yscale('log')  # Set y axis to log scale

    plt.title(f"Total Carbon vs. Inference Frequency\nWorkload: {workload_choice}")
    plt.xlabel("Inference Frequency (inferences/year)")
    plt.ylabel("Total Carbon Footprint (gCO₂-eq)")
    plt.legend()
    plt.show()

    # -----------------------------------------
    # Part 1.5: Plot total carbon vs. number of inferences
    # -----------------------------------------
    
    inferences = np.logspace(np.log10(MIN_INF_FREQ * MIN_LIFETIME), np.log10(MAX_INF_FREQ *MAX_LIFETIME), 50)  # Log scale from 1 to 3650 inferences
    plt.figure(figsize=(6, 4))
    
    for system in system_specs:
        total_carbons = []
        for inf in inferences:
            lt = inf / inf_freq  # Calculate lifetime from inferences
            _, _, total = compute_total_carbon(system, workload_choice, lt,
                                               inf_freq, carbon_intensity,
                                               core_freq)  # Pass core frequency
            total_carbons.append(total)
        plt.plot(inferences, total_carbons, label=system, color=system_colors[system])

    # Add a vertical dashed line at the x value of inf_freq * lifetime_yrs
    plt.axvline(x=inf_freq * lifetime_yrs, color='gray', linestyle='--', linewidth=1)

    plt.xscale('log')  # Set x axis to log scale
    plt.yscale('log')  # Set y axis to log scale

    plt.title(f"Total Carbon vs. Number of Inferences\nWorkload: {workload_choice}")
    plt.xlabel("Number of Inferences")
    plt.ylabel("Total Carbon Footprint (gCO₂-eq)")
    plt.legend()
    plt.show()

    # -----------------------------------------
    # Part 2: 2D region map (lifetime vs. inferences)
    # -----------------------------------------
    # Create a fine grid for smooth contours
    lifetime_grid = np.logspace(np.log10(MIN_LIFETIME), np.log10(MAX_LIFETIME), 200)   # increase resolution on log scale
    inf_freq_grid = np.logspace(np.log10(MIN_INF_FREQ), np.log10(MAX_INF_FREQ), 200)    # increase resolution on log scale
    
    X, Y = np.meshgrid(lifetime_grid, inf_freq_grid)

    systems = list(system_specs.keys())  # [Serv, Qerv, Herv, Nerv]
    
    # Calculate carbon values for each system
    carbon_values = np.ones((len(systems), len(inf_freq_grid), len(lifetime_grid))) *np.inf
    for sys_idx, sys_name in enumerate(systems):
        for i, freq in enumerate(inf_freq_grid):
            for j, lt in enumerate(lifetime_grid):
                _, _, total = compute_total_carbon(sys_name, workload_choice, lt,
                                               freq, carbon_intensity,
                                               core_freq)  # Pass core frequency
                carbon_values[sys_idx, i, j] = total

    plt.figure(figsize=(6, 4))
    best_indices = np.argmin(carbon_values, axis=0)
    
    # Create contours where systems intersect
    for i in range(len(systems)):
        for j in range(i+1, len(systems)):
            # Find where systems i and j intersect and either is the best index
            diff = carbon_values[i] - carbon_values[j]
            mask = (best_indices == i) | (best_indices == j)
            masked_diff = np.ma.masked_where(~mask, diff)
            plt.contour(X, Y, masked_diff, levels=[0], colors='black', linestyles='-', linewidths=1)
    
    # Color the regions
    plt.pcolormesh(X, Y, best_indices, shading='auto', alpha=0.3, cmap=ListedColormap([system_colors[sys] for sys in systems][np.min(best_indices):np.max(best_indices)+1]))
    
    # Add legend
    handles = [plt.Line2D([0], [0], marker='o', color='w', label=sys, markersize=10, markerfacecolor=system_colors[sys]) for sys in systems]
    plt.legend(handles=handles, title="Systems")
    
    # Plot a marker at the currently selected lifetime & inference frequency
    plt.plot([lifetime_yrs], [inf_freq], 'o', markersize=8)
    
    plt.xscale('log')
    plt.yscale('log')
    plt.title("Best System Choice by (Lifetime, Inference Freq)")
    plt.xlabel("Lifetime (years)")
    plt.ylabel("Inferences per Year")
    plt.show()
    # # New plot for core frequency sweep
    # core_freq_values = np.linspace(MIN_CORE_FREQ, MAX_CORE_FREQ, 50)  # Define core frequency range
    # total_carbon_core_freq = np.zeros((len(core_freq_values), len(systems)))

    # for k, core_freq in enumerate(core_freq_values):
    #     for sys_idx, sys_name in enumerate(systems):
    #         _, _, total = compute_total_carbon(sys_name, workload_choice, lt,
    #                                            inf_freq, carbon_intensity,
    #                                            core_freq)  # Pass core frequency
    #         total_carbon_core_freq[k, sys_idx] = total

    # plt.figure(figsize=(6, 4))
    # for sys_idx, sys_name in enumerate(systems):
    #     plt.plot(core_freq_values, total_carbon_core_freq[:, sys_idx], label=sys_name)

    # plt.title("Total Carbon Footprint vs. Core Frequency for Each System")
    # plt.xlabel("Core Frequency (MHz)")
    # plt.ylabel("Total Carbon Footprint (gCO₂-eq)")
    # plt.legend()
    # plt.xscale('linear')
    # plt.yscale('log')
    # plt.show()
    
    

In [40]:
# -----------------------------------------
# 2) Set up interactive widgets
# -----------------------------------------
workload_dropdown = widgets.Dropdown(
    options=execution_time.keys(),
    value="Food Spoilage Detection",
    description="Workload:",
)

lifetime_slider = widgets.FloatSlider(
    value=1.0,
    min=MIN_LIFETIME,
    max=MAX_LIFETIME,
    step=0.5,
    description='Lifetime (yrs)'
)

inf_freq_slider = widgets.BoundedFloatText(
    value=1.0,
    min=MIN_INF_FREQ,
    max=MAX_INF_FREQ,
    step=1.0,
    description='Inf. per Year'
)

power_source_dropdown = widgets.Dropdown(
    options=carbon_intensity_values.keys(),
    value=list(carbon_intensity_values.keys())[0],
    description="Power Source:",
)

carbon_intensity_box = widgets.Text(
    value=carbon_intensity_values[power_source_dropdown.value],
    description="Carbon Int. (CO₂/unit energy)",
)

carbon_intensity_box.disabled = True
def update_carbon_intensity(*args):
    if power_source_dropdown.value == "Custom":
        carbon_intensity_box.disabled = False
    else:
        carbon_intensity_box.value = carbon_intensity_values[power_source_dropdown.value]
        carbon_intensity_box.disabled = True

power_source_dropdown.observe(update_carbon_intensity, 'value')

core_freq_box = widgets.BoundedFloatText(
    value=10000.0,
    min=1000.0,
    max=100000.0,
    step=100.0,
    description='Core Freq'
)

ui = widgets.VBox([
    workload_dropdown,
    lifetime_slider,
    inf_freq_slider,
    power_source_dropdown,
    carbon_intensity_box,
    core_freq_box
])

out = widgets.interactive_output(
    plot_carbon_vs_lifetime,
    {
        'workload_choice': workload_dropdown,
        'lifetime_yrs': lifetime_slider,
        'inf_freq': inf_freq_slider,
        'carbon_intensity': carbon_intensity_box,
        'core_freq': core_freq_box,
    }
)

display(ui, out)

Output()